# scvi prep

* Going to perform scvi integration to show results are comparable to harmony integration. Xiaohan will perform integration, however scvi needs to be done on raw data so preparing raw object here (take original raw object but then remove cells that were excluding during cleaning and transfer relevant metadata)

In [ ]:
import scanpy as sc
import os
import pandas as pd